## Setup

In [ ]:
# !pip install openai --quiet
# !pip install fitz
# !pip install pymupdf

## Imports & API Key

In [ ]:
import json
from openai import OpenAI
import openai
import requests
from io import BytesIO
import os
from google.colab import userdata
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionXLPipeline
from peft import get_peft_model, LoraConfig
import torch
import base64
import requests
from PIL import Image as PILImage
from IPython.display import display
import fitz
import fitz  # PyMuPDF
import openai
import re

In [ ]:
# get
ai_token = userdata.get('ai_token')
client = openai.OpenAI(api_key=ai_token)
# print(ai_token)


In [ ]:
# # List all models
# models = client.models.list()

# # Print model IDs
# for model in models.data:
#     print(model.id)

## Load JSON File

In [ ]:
# Load your model description from JSON
json_path = "models_fibrillar interfaces.json"  # change to your file
with open(json_path, "r") as f:
    model_data = json.load(f)

## Load PDF

In [ ]:
import fitz  # PyMuPDF
import openai

def extract_pdf_summary(pdf_path):
    # === Step 1: Extract Text ===
    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()

    # === Step 3: Prompt GPT to extract core info ===
    title_prompt = f"""
    You are analyzing a scientific article. From the text below, extract:
    1. A 4-6 words clean and accurate title that captures the paper’s topic
    2. The final human-engineered object or system (e.g., "Train", "Sensor", "Wing", "Nose Cone", "Surface Coating")

    Be concise. Output JSON like this:
    {{
      "title": "...",
      "organism_": "..."
    }}

    Article:
    \"\"\"
    {text}
    \"\"\"
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert in summarizing scientific papers for visual infographic generation."},
            {"role": "user", "content": title_prompt}
        ],
        temperature=0.2
    )

    answer = response.choices[0].message.content
    answer = answer.strip().removeprefix("```json").removesuffix("```").strip()

    return answer


In [ ]:
pdf_path = 'fibrillar interfaces.pdf'
title_pdf = extract_pdf_summary(pdf_path)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
title_pdf = json.loads(title_pdf)
title_pdf

## Prompt to Generate Summary from JSON

In [ ]:
def generate_summary_prompt(model_data, filename="Untitled"):

    if isinstance(model_data, dict) and "models" in model_data:
        models = model_data["models"]
        num = len(models)

    if num > 1:
      pdf = True
      print("true")
      base_title = os.path.splitext(os.path.basename(filename))[0].replace("_", " ").title()
      prompt = f"""
You are given a JSON file that contains **multiple biological or technical models** under a key called "models".

Each model has a `title`, an `organism`, and a list of `functional_components`.

You should return a single structured JSON object summarizing **all models** as a multi-layer system.

The output must include:
- title: Use the filename as the global title (e.g., "{base_title}")
- organism : object or organism based on the title
- layers: Each model becomes a layer, derived from its first functional component.

Each layer must include:
  - name
  - function: from the model's first functional component
  - description
  - caption: a condensed version of the function in ≤10 words
  - scale: from the component
  - visual: what to draw based on model + function

Sort the layers by descending scale string (but do not convert units).

Here is the data:
{json.dumps(model_data, indent=2)}
"""
    elif num == 1:
        pdf = False
        print("False")
        prompt = f"""
You are given the JSON below that describes a **single** biological or technical system.

Extract a structured summary as a clean, valid JSON object with the following keys:
- title
- organism
- layers: a list of objects sorted from largest to smallest based on the scale string (but do not convert the units), make sure it's sorted from the largest to the smallest

Each layer should include:
  - name
  - function
  - description
  - caption (a condensed version of the function in ≤10 words)
  - scale
  - visual (a short instruction on what to draw)

Here is the data:
{json.dumps(model_data, indent=2)}
"""
    return prompt


In [ ]:
prompt = generate_summary_prompt(model_data,json_path)

## Call ChatGPT-4o to Summarize

In [ ]:
#client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You extract and organize JSON content for scientific diagram generation."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.2
)

raw_output = response.choices[0].message.content

# Remove the triple backticks and optional 'json'
cleaned = raw_output.strip().removeprefix("```json").removesuffix("```").strip()


# Parse into Python dict
data = json.loads(cleaned)
if type(data) == list:
  data = data[0]

#summary = json.loads(response.choices[0].message.content)
# print(data)


print(json.dumps(data, indent=2, ensure_ascii=False))

## Generate Final Prompt for Image Generation

In [ ]:
def generate_infographic_prompt(summary, pdf= True):

    title =  title_pdf['title'] if pdf else summary['title']
    print(title_pdf['organism_'])

    organism = title_pdf["organism_"] if pdf else summary['organism']
    layers = summary["layers"]

    prompt = f"""Create an isometric scientific infographic titled “{title}”, featuring vertically exploded layers aligned on a central axis, evenly spaced, and viewed from a 30° isometric camera angle on both X and Y axes.

Layout & Visual Style:
- Background: Flat grey (#CCCCCC) with a light isometric grid
- No shadows, no icons, no decorative elements
- Title: Bold, centered at the top: {title}

Top Layer (Organism/System):
- A photo-realistic rendering of the {organism}, viewed from above, centered.
- No background environment—only the grid.

Lower Layers (Functional Components, in black-and-white architectural line style):"""

    for layer in layers:
        prompt += f"""

- “{layer['name']}”
    - Left caption: “{layer['name']} – {layer['caption']}”
    - Right label: {layer['scale']}
    - Visual: {layer['visual']}"""

    prompt += """

Final Visual Instructions:
- Top layer: Full-color, photo-realistic
- Lower layers: Black-and-white line drawings
- Vertical black ruler on the right, aligned to scales
- Captions and labels outside the visuals, no overlaps
- Maintain blueprint-style visual consistency"""

    return prompt

final_prompt = generate_infographic_prompt(data, pdf = False)
print(final_prompt)


In [ ]:
final_prompt_test = """
Create an isometric scientific infographic titled “Lotus Leaf Superhydrophobic Surface (Droplet Impalement-Resistant)” featuring a vertically exploded diagram with exactly five layers. The layout should be clean, blueprint-style, with no decorative elements, and viewed from a 30° isometric camera angle on both X and Y axes. All components must be aligned on a central vertical axis and spaced evenly.

Layout & Style:
* Canvas format: Portrait (e.g., 1024×1536 or taller)
* Background: Flat neutral grey (#CCCCCC) with a subtle isometric grid
* No shadows, icons, or decorative elements
* Title: Bold black font, centered at the top
* Captions: Aligned to the left of each layer
* Scale labels: Aligned to the right of each layer
* Style: Top layer photo-realistic; all lower layers black-and-white architectural line drawings

Top Layer (Organism):
* Visual: A photo-realistic rendering of a round Lotus Leaf (Nelumbo) seen from above with visible green veins and a large glossy water droplet centered on it (strong reflection highlight on droplet)
* Label (right): “Lotus Leaf (Nelumbo)”

Layer 1 – Papillae
* Caption (left): “Papillae – Micro-bumps reduce contact”
* Label (right): “1–5 µm”
* Visual: A regular grid of dome-shaped micro-bumps, resembling bubble wrap

Layer 2 – Air Layer
* Caption (left): “Air Layer – Trapped gas reduces adhesion”
* Label (right): “1–5 µm”
* Visual: A flat grid plane with a centered water droplet cross-section and a visible gas layer underneath

Layer 2 – Nanohairs
* Caption (left): “Nanohairs – Texture enhances repellency”
* Label (right): “200 nm”
* Visual: A forest of vertical nano-needle hairs densely packed

Layer 2 – Wax Crystals
* Caption (left): “Wax Crystals – Low surface energy repels water”
* Label (right): “100 nm”
* Visual: A dense field of irregular, frost-like or grainy wax crystals

"""

## Generate Image via OpenAI API

In [ ]:
result = client.images.generate(
    model="gpt-image-1",
    prompt= final_prompt,
    n=1,
    size="1024x1536"
)

image_base64 = result.data[0].b64_json
image_bytes = base64.b64decode(image_base64)
image = PILImage.open(BytesIO(image_bytes))
display(image)

In [ ]:
# Save the image to a file
# with open("otter.png", "wb") as f:
#     f.write(image_bytes)